In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neptune
from models import create_compound_model_from_info, create_pytorch_model_from_info, MultiHead
import json
import torch
import os
from utils_neptune import check_if_path_in_struc, get_sub_struc_from_path

NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='


In [ ]:
run_id = 'RCC-2925'

run = neptune.init_run(project='revivemed/RCC',
    api_token= NEPTUNE_API_TOKEN,
    with_id=run_id,
    mode="read-only")   
run_struc= run.get_structure()

In [5]:

task_id = 'Optimized_NIVO-OS ADV EVER-OS__133_finetune'
local_dir = os.path.expanduser('~/saved_models')
model_dir = f'{local_dir}/{run_id}/{task_id}'
components_dir = f'{local_dir}/{run_id}/{task_id}/components'
os.makedirs(components_dir, exist_ok=True)


substruc = get_sub_struc_from_path(run_struc,f'{task_id}/models')

for key in substruc.keys():
    if 'info' in key:
        run[f'{task_id}/models/{key}'].download(f'{components_dir}/components/{key}.json')
    elif 'state' in key:
        run[f'{task_id}/models/{key}'].download(f'{components_dir}/components/{key}.pt')


Fetching file...: 0 [00:00, ?/s]

In [6]:
os.listdir(save_dir)

['Cox_EVER OS_info.json',
 'encoder_info.json',
 'Cox_NIVO OS_info.json',
 'Cox_NIVO OS_state.pt',
 'encoder_state_dict.pt',
 'Cox_EVER OS_state.pt']

In [ ]:
# load the relavent encoder info and head info
head_key = 'Cox_NIVO'
model_files = os.listdir(components_dir)

encoder_info = None
head_info = None
encoder_state = None
head_state = None

for f in model_files:
    if ('encoder' in f):
        if 'info' in f:
            encoder_info = json.load(open(f'{components_dir}/{f}'))
        elif 'state' in f:
            encoder_state = torch.load(f'{components_dir}/{f}')
    if (head_key in f):
        if 'info' in f:
            head_info = json.load(open(f'{components_dir}/{f}'))
            # head_name = f.replace('_info.json', '')
        elif 'state' in f:
            head_state = torch.load(f'{components_dir}/{f}')
            
            
if encoder_info is not None and head_info is not None:            
    model = create_compound_model_from_info(encoder_info=encoder_info, 
                                            head_info= head_info,
                                            encoder_state_dict=encoder_state,
                                            head_state_dict=head_state)
    
    model.save_info(model_dir, f'VAE {head_key} info.json')
    model.save_state_to_path(model_dir, f'VAE {head_key} state.pt')

    skmodel = create_pytorch_model_from_info(full_model=model)